In [ ]:
!pip install openai groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.0 MB/s eta 0:00:00
Conversation History after 3 turns + summarization:
[{'role': 'system', 'content': "Summary so far: A user initiated a conversation with an assistant to inquire about their booking. The assistant responded by requesting the user's booking ID to proceed. The user provided the booking ID, which is 12345. The conversation is ongoing, with the assistant likely to provide information about the booking next."}, {'role': 'user', 'content': "Yes, it's 12345."}]
Extracted Info: {"age":29,"email":"alice@gmail.com","location":"New York","name":"Alice Johnson","phone":"9876543210"}


In [ ]:
import os
from openai import OpenAI
from groq import Groq

# ✅ Setup Groq client (OpenAI-compatible)
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# --------------------------
# Task 1: Conversation Management
# --------------------------
conversation_history = []
summary_counter = 0
K = 3  # summarize after every 3 turns

def add_message(role, content):
    global summary_counter, conversation_history
    conversation_history.append({"role": role, "content": content})
    summary_counter += 1

    # Perform summarization every K turns
    if summary_counter % K == 0:
        summarize_conversation()

def summarize_conversation():
    global conversation_history
    text = "\n".join([f"{m['role']}: {m['content']}" for m in conversation_history])
    prompt = f"Summarize the following conversation in 3-4 sentences:\n{text}"

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    summary = response.choices[0].message.content
    # Replace history with summary + keep last 1 message
    conversation_history = [{"role": "system", "content": f"Summary so far: {summary}"},
                            conversation_history[-1]]

def truncate_history_by_turns(n=4):
    global conversation_history
    conversation_history = conversation_history[-n:]

def truncate_history_by_chars(limit=200):
    global conversation_history
    text = "\n".join([m['content'] for m in conversation_history])
    if len(text) > limit:
        conversation_history = [{"role": "system", "content": text[-limit:]}]

# --------------------------
# Task 2: JSON Schema Extraction
# --------------------------
functions = [
    {
        "name": "extract_user_info",
        "description": "Extract user details from chat",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "phone": {"type": "string"},
                "location": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": ["name", "email", "phone", "location", "age"]
        }
    }
]

def extract_info(chat_text):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": chat_text}],
        functions=functions,
        function_call={"name": "extract_user_info"}
    )
    return response.choices[0].message.function_call.arguments

# --------------------------
# DEMONSTRATION
# --------------------------
# Task 1 Demo
add_message("user", "Hi, I want to know about my booking.")
add_message("assistant", "Sure, can you give me your booking ID?")
add_message("user", "Yes, it's 12345.")  # → summarization happens here

print("Conversation History after 3 turns + summarization:")
print(conversation_history)

# Task 2 Demo
sample_chat = "My name is Alice Johnson, email alice@gmail.com, phone 9876543210, I live in New York, and I’m 29 years old."
print("Extracted Info:", extract_info(sample_chat))


Conversation History after 3 turns + summarization:
[{'role': 'system', 'content': "Summary so far: A user initiated a conversation to inquire about their booking. The assistant responded by requesting the user's booking ID to access the relevant information. The user provided the booking ID, which is 12345, to help the assistant locate their booking details. The conversation is ongoing, with the assistant likely to provide an update on the booking next."}, {'role': 'user', 'content': "Yes, it's 12345."}]
Extracted Info: {"age":29,"email":"alice@gmail.com","location":"New York","name":"Alice Johnson","phone":"9876543210"}
